## Twitter Streaming App
This is the code to run a Stream Listener to capture Twitter feeds based on a specific keyword list (could be hashtags)<br>
<b>Each tweet is analysed for sentiment and emotion in realtime <br>
   and the tweet+analysis+goe-coordinates are saved on a Cloudant database for further processing/display etc</b>



In [ ]:
import pip
pip.__version__
!python3 -m pip install --user tweepy
!python3 -m pip install --user cloudant
!pip install --upgrade watson-developer-cloud

The following cell contains code to connect to Twitter
requires Twitter API keys


def <b>connect_to_twitter():</b><br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;import tweepy<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;consumerKey = 'xxxxxxxxxxxxxxxxxxx'<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;consumerSecret = 'xxxxxxxxxxxxxxxx'<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;accessToken = 'xxxxxxxxxxxxxxxxxxxx'<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;accessTokenSecret = 'xxxxxxxxxxxxxxxxxxxxxx'<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;auth = tweepy.OAuthHandler(consumerKey, consumerSecret)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;auth.set_access_token(accessToken, accessTokenSecret)<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;return tweepy.API(auth_handler=auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)  <br>


In [ ]:
# The code was removed by Watson Studio for sharing.

In [9]:
#from twitterconnect import connect_to_twitter
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

The following cell contains the credentials to access Watson NLU API

#Watson NLU credentials<br>
credentials_NLU = {<br>
  'url':'https://gateway.watsonplatform.net/natural-language-understanding/api',<br>
  'username':'xxxxxxxxxxxxxxxxxxxx',<br>
  'password':'xxxxxxxxxxxxxxxxxxxxxxxx'<br>
}<br>


In [28]:
# The code was removed by Watson Studio for sharing.

In [44]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions, EmotionOptions

NLU_service = NaturalLanguageUnderstandingV1(
    version='2018-03-16',
    ## url is optional, and defaults to the URL below. Use the correct URL for your region.
    # url='https://gateway.watsonplatform.net/natural-language-understanding/api',
    username=credentials_NLU['username'],
    password=credentials_NLU['password'])


The folowing cell contains Cloudant credentials<br>
#Cloudant credentials<br>
credentials_cloudant = {<br>
  'password':"""xxxxxxxxxxxxxxxxxxxxxx""",<br>
  'custom_url':'https://xxxxxxxxxxxxxxxxxxxxxxxx-bluemix.cloudant.com',<br>
  'username':'xxxxxxxxxxxxxxxxxxxxxxxxx-bluemix',<br>
  'url':'https://undefined'<br>
}

In [1]:
# The code was removed by Watson Studio for sharing.

In [ ]:
#connect to Cloudant directly 
from cloudant.client import Cloudant
from cloudant.error import CloudantException
from cloudant.result import Result, ResultByKey

serviceUsername = credentials_cloudant['username']
servicePassword = credentials_cloudant['password']
serviceURL = credentials_cloudant['custom_url']
client = Cloudant(serviceUsername, servicePassword, url=serviceURL)
client.connect()

In [15]:
#connect via Spark driver

spark = SparkSession\
    .builder\
    .appName("Cloudant Storage for Tweets")\
    .config("cloudant.host",credentials_cloudant['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_cloudant['username'])\
    .config("cloudant.password",credentials_cloudant['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

#df = spark.read.load("shake_classification", "org.apache.bahir.cloudant")


In [47]:
#Cloudant database management

def create_tweets_database(db_name):
    database = client.create_database(db_name, throw_on_exists=False)
    if database.exists():
        print("{} database successfully created".format(db_name))
        return database
    else:
        return None
    
def delete_tweets_database(db_name):
    try :
        client.delete_database(db_name)
    except CloudantException:
        print("There was a problem deleting {}.\n".format(db_name))
    else:
        print("{} successfully deleted.\n".format(db_name))


In [62]:
#Tweets Streaming Classes

def get_center_coordinates(b_box):
    return [sum([b_box[0][x][0] for x in range(4)])/4, sum([b_box[0][x][1] for x in range(4)])/4]


def create_JSON_record(status):
    coords=None
    country=""
    if (status.coordinates != None):
        coords = status.coordinates['coordinates']
    if (status.place != None):
        coords = get_center_coordinates(status.place.bounding_box.coordinates)
        country = status.place.country_code
    try:
        response = NLU_service.analyze(text=status.text, 
            features=Features(sentiment=SentimentOptions(), emotion=EmotionOptions())).get_result()
        tweet_emotion = response['emotion']['document']['emotion']
        tweet_sentiment = response['sentiment']['document']
    except:
        tweet_emotion = {}
        tweet_sentiment = {}        
    json_doc = {
        "dateCreated": status.created_at.isoformat().split('T')[0],
        "timeCreated": status.created_at.isoformat().split('T')[1],
        "country": country,
        "coords": coords,
        "sentiment": tweet_sentiment,
        "emotion": tweet_emotion,
        "text": status.text}
    return json_doc


class Listener(StreamListener):
    def __init__(self, tweets_database):
        super(Listener, self).__init__()
        self.tweets_database = tweets_database
        self.counter=0
        
    def on_status(self, status):
        if (status.place != None) or (status.coordinates != None):
            self.counter +=1
            json_doc = create_JSON_record(status)
            print("#{:d} date/time: {}/{} country: {} coords: {}". format(self.counter,
                                    json_doc["dateCreated"],json_doc["timeCreated"], json_doc["country"], json_doc["coords"]))
            print("sentiment: {} emotion: {}". format(json_doc["sentiment"], json_doc["emotion"]))
 
            print("text: ", json_doc["text"])
            new_record = self.tweets_database.create_document(json_doc)
        return True

    def on_error(self, status_code):
        if status_code == 420:
            #returning False in on_data disconnects the stream
            return False
        
        
class TwitterStreamer():  #Class for streaming and processing live tweets.
    def __init__(self, filter_list):
        self.filter_list = filter_list
        pass

    def stream_tweets(self, tweets_database):
        #This handles Twitter authetification and the connection to Twitter Streaming API
        listener = Listener(tweets_database)
        stream = Stream(twapi.auth, listener)
        #This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=self.filter_list)

## MAIN LOOP

In [63]:
#Main streaming procedure       
    
hash_tag_list = ['brexit']
tweets_database = create_tweets_database("brexit_tweets")
twapi=connect_to_twitter()        
        
if __name__ == '__main__':
    ts = TwitterStreamer(hash_tag_list)
    try:
        ts.stream_tweets(tweets_database)
    except KeyboardInterrupt:
        print("exited early")
    #client.disconnect()


brexit_tweets database successfully created
#1 date/time: 2018-10-18/10:58:26 country: GB coords: [-1.449612, 53.3831645]
sentiment: {'score': 0.784187, 'label': 'positive'} emotion: {'sadness': 0.109548, 'joy': 0.62094, 'anger': 0.144433, 'disgust': 0.054205, 'fear': 0.084297}
text:  This is important! So long beef trade deal! #brexit #peoplesvote
#2 date/time: 2018-10-18/10:59:14 country: GB coords: [-3.0033304999999997, 53.632616]
sentiment: {'score': -0.541009, 'label': 'negative'} emotion: {'sadness': 0.662423, 'joy': 0.054157, 'anger': 0.132683, 'disgust': 0.189046, 'fear': 0.187289}
text:  @SkyNewsPolitics @SkyNews @Jacob_Rees_Mogg "I think it's a poorly thought-through idea"   Unlike the Brexit referen… https://t.co/0vuAtaTgZl
#3 date/time: 2018-10-18/10:59:47 country: GB coords: [-0.3289185, 52.5495755]
sentiment: {'score': 0.0, 'label': 'neutral'} emotion: {'sadness': 0.571557, 'joy': 0.125085, 'anger': 0.069175, 'disgust': 0.269251, 'fear': 0.195669}
text:  #Kool_Satsuma &am

ProtocolError: ('Connection broken: IncompleteRead(0 bytes read)', IncompleteRead(0 bytes read))